In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df, hue = "Outcome")

In [ ]:
import numpy as np
X = df.drop('Outcome', axis = 1).values
y = df['Outcome']

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
Xtrain = torch.FloatTensor(Xtrain)
Xtest = torch.FloatTensor(Xtest)
ytrain = torch.LongTensor(ytrain.values)
ytest = torch.LongTensor(ytest.values)

In [ ]:
class ANN_Model(nn.Module):
    def __init__(self, input_features = 8, hidden1 = 20, hidden2 = 20, out_features = 2):
        super().__init__()
        self.f_connected1 = nn.Linear(input_features, hidden1)
        self.f_connected2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        x = F.relu(self.f_connected1(x))
        x = F.relu(self.f_connected2(x))
        x = self.out(x)
        return x

In [ ]:
torch.manual_seed(20)
model = ANN_Model()

In [ ]:
model.parameters

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
epochs = 5000
final_losses = []
for i in range(epochs):
    i += 1
    y_pred = model.forward(Xtrain)
    loss = loss_function(y_pred, ytrain)
    final_losses.append(loss)
    if i%10 == 1:
        print("Epoch number: {} and the loss: {}".format(i, loss.item()))
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(range(epochs), final_losses)
plt.ylabel("Loss")
plt.xlabel("Epochs")

In [ ]:
predictions = []
with torch.no_grad():
    for i, data in enumerate(Xtest):
        y_pred = model(data)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

In [ ]:
predictions

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, predictions)
cm

In [ ]:
plt.figure(figsize = (10, 6))
sns.heatmap(cm, annot = True)
plt.xlabel("Actual values")
plt.ylabel("Predicted values")

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(ytest, predictions))

In [ ]:
torch.save(model, 'diabetes.pt')

In [ ]:
model = torch.load('diabetes.pt')
model.eval